# Claim-based Question-Answer Hallucination Detection

Import necessary packages.

In [1]:
import time
from uqlm.longform.black_box import ClaimQAScorer
from uqlm import BlackBoxUQ

#### Load LLM and device

In [2]:
# from langchain_google_vertexai import ChatVertexAI
# llm = ChatVertexAI(model="gemini-1.5-flash")

from dotenv import load_dotenv, find_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv(find_dotenv())
llm = AzureChatOpenAI(
    deployment_name="gpt-4o-mini",
    openai_api_type="azure",
    openai_api_version="2024-02-15-preview",
    temperature=1,  # User to set temperature
)

In [3]:
import torch

# Set the torch device
if torch.cuda.is_available():  # NVIDIA GPU
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # macOS
    device = torch.device("mps")
else:
    device = torch.device("cpu")  # CPU
print(f"Using {device.type} device")

Using cuda device


#### Setup Prompts and Black Box Scorer

In [4]:
prompts = [
    "write a paragraph about Paul McCartney",
    "write a paragraph about John Lennon"
]

In [5]:
bb_scorer = BlackBoxUQ(
    llm=llm,
    max_calls_per_min=500,  # set value to avoid rate limit error
    device=device,
    scorers=["exact_match"],
)

#### Claim-QA class

There are three methods that can be used to compute Claim-QA score.
- `generate_and_score`: If you only have prompts, call this method generate long response, decompose that response into factoids, then generate questions for each factoids, and compute question-level, factoid-level, and response-level scores.
- `score`: If you already generated long response, call this method
- `evaluate`: If you already have decomposed long responses into factoids, call this method

##### 1. `generate_and_score` method

In [6]:
claim_qa = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500, num_questions=2)
start_time = time.time()
result = await claim_qa.generate_and_score(prompts=prompts)

print(f"Computation time: {time.time() - start_time} seconds")

Generated questions:  ['### Who is a famous musician known for being part of The Beatles and a successful solo artist?', '### What nationality and profession is Paul McCartney known for?', "### What is Paul McCartney's occupation?", "### What is Paul McCartney's professional role in music?", '### Who is best known as one of the founding members of The Beatles?', '### Which band is widely recognized as one of the most influential in the history of popular music?', '### What is the birth date of Paul McCartney?', '### In which city was Paul McCartney born?', '### In which country is Liverpool located?', '### At what age did Paul McCartney first show signs of musical talent?', '### At what age did Paul McCartney form his first band?', "### Which musician is known for significantly contributing to the band's remarkable discography with The Beatles?", '### Who is known for writing the iconic song "Yesterday"?', '### Who is known for writing the iconic song "Hey Jude"?', '### Who is the song

In [14]:
llm.temperature = 1.5
llm.invoke(

AIMessage(content='Paul McCartney.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 47, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CTX7XRPdEzYZm7gjtxLGJtGygiLKh', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run--c8b71b2a-601e-4f28-91b7-25f94680eb38-0', usage_metadata={'input_tokens': 47, 'output_tokens': 5, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [15]:
prompts = ["Consider the following question and answer with as few words as possible:\n    \n    Who is a famous musician known for being part of The Beatles and a successful solo artist?\n\n    Now your answer is:\n    "]
bb_result = await bb_scorer.generate_and_score(prompts=prompts, num_responses=10)

Output()

Output()

In [17]:
bb_result.to_df()["sampled_responses"][0]

['Paul McCartney.',
 'Paul McCartney.',
 'Paul McCartney.',
 'Paul McCartney.',
 'Paul McCartney.',
 'Paul McCartney',
 'Paul McCartney.',
 'Paul McCartney.',
 'Paul McCartney.',
 'Paul McCartney.']

In [7]:
result.to_df()

,prompt,response,response_scores_exact_match,factoid_scores_exact_match
0,write a paragraph about Paul McCartney,"Paul McCartney is an iconic British musician, ...",0.685714,"[0.6, 0.7, 1.0, 0.8, 1.0, 0.3, 0.5, 0.9, 0.8, ..."
1,write a paragraph about John Lennon,"John Lennon was a groundbreaking musician, pea...",0.607143,"[0.6, 1.0, 1.0, 1.0, 1.0, 0.5, 0.1, 0.6, 0.7, ..."


#### 2. `score` method

In [8]:
claim_qa2 = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500)
start_time = time.time()
result2 = await claim_qa2.score(prompts=claim_qa.prompts, responses=claim_qa.responses)
print(f"Computation time: {time.time() - start_time} seconds")

Generated questions:  ['### What are some notable accomplishments of Paul McCartney in the music industry?', "### What can you tell me about Paul McCartney's musical career and contributions?", '### What role did Paul McCartney play in the music industry?', '### What key contributions is Paul McCartney recognized for in the music industry?', '### When was Paul McCartney born?', '### Where was Paul McCartney born?', "### What early experiences contributed to Paul McCartney's development as a musician?", "### What impact did Paul McCartney's collaboration with another musician have on music history?", '### What are some of the notable achievements of Paul McCartney during his time with The Beatles?', '### What are some of the notable songs written by Paul McCartney during his time with The Beatles?', '### What are some notable songs that Paul McCartney wrote during his time with The Beatles?', '### What are some of the iconic songs associated with Paul McCartney and The Beatles?', '### W

In [9]:
result2.to_df()

,prompt,response,response_scores_exact_match,factoid_scores_exact_match
0,write a paragraph about Paul McCartney,"Paul McCartney is a renowned English musician,...",0.160000,"[0.0, 0.0, 0.2, 0.2, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
1,write a paragraph about John Lennon,"John Lennon was an iconic British musician, so...",0.348148,"[1.0, 0.0, 0.0, 1.0, 0.4, 0.6, 1.0, 1.0, 1.0, ..."


#### 3. `evaluate` method

In [7]:
claim_qa3 = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500)
start = time.time()
result3 = await claim_qa3.evaluate(prompts=claim_qa.prompts, responses=claim_qa.responses, factoids=claim_qa.factoids)
stop = time.time()
# print(f"Computation time: {time.time() - start_time} seconds")

Generated questions:  ['### Who is considered a legendary musician known for his influential contributions to music?', '### What is the nationality and profession of Paul McCartney?', "### What is Paul McCartney's profession?", "### What is Paul McCartney's primary profession in the music industry?", '### Who is best known as one of the co-founders of The Beatles?', '### On what date was Paul McCartney born?', '### In which city was Paul McCartney born?', '### Where was Paul McCartney born?', "### What is true about Paul McCartney's musical ability during his childhood?", '### What instrument did Paul McCartney learn to play?', '### What instrument did Paul McCartney learn to play?', '### Who is credited with revolutionizing popular music in the 1960s?', '### Which two members of The Beatles are known for their collaboration as songwriters?', '### Which two members of The Beatles worked together on musical projects?', '### Who collaborated with Ringo Starr?', '### What significant cont

In [8]:
claim_qa3.response_fact_questions

[['\n    Consider the following question and answer with as few words as possible, e.g. only a name, place, or thing:\n    \n    Who is considered a legendary musician known for his influential contributions to music?\n\n    Now your answer is:\n    ',
  '\n    Consider the following question and answer with as few words as possible, e.g. only a name, place, or thing:\n    \n    What is the nationality and profession of Paul McCartney?\n\n    Now your answer is:\n    ',
  "\n    Consider the following question and answer with as few words as possible, e.g. only a name, place, or thing:\n    \n    What is Paul McCartney's profession?\n\n    Now your answer is:\n    ",
  "\n    Consider the following question and answer with as few words as possible, e.g. only a name, place, or thing:\n    \n    What is Paul McCartney's primary profession in the music industry?\n\n    Now your answer is:\n    ",
  '\n    Consider the following question and answer with as few words as possible, e.g. only 

In [9]:
claim_qa3.response_fact_questions_responses

[[['Jimi Hendrix', 'British musician.', 'Musician', 'Musician', 'John Lennon'],
  ['June 18, 1942',
   'Liverpool',
   'Liverpool',
   'Talented musician.',
   'Bass guitar'],
  ['Bass guitar',
   'The Beatles',
   'Lennon and McCartney',
   'John Lennon and Paul McCartney.',
   'Paul McCartney'],
  ['Beatles, Wings, solo career, songwriting.',
   'Revolver',
   "Sgt. Pepper's Lonely Hearts Club Band",
   '"Revolver"',
   'Formed Wings.'],
  ['1970',
   'Wings',
   '"Maybe I\'m Amazed," "Live and Let Die," "Band on the Run," "My Love."',
   'The Beatles',
   'Melody'],
  ['Global.',
   'Rock, Pop, Classical.',
   'Rock; example: Indie rock.',
   'Pop, rock, classical, electronic.',
   'Global fame'],
  ['Grammy Awards', 'Paul McCartney', 'The Beatles', 'Philanthropy']],
 [['John Lennon', 'Musician', 'Musician', 'The Beatles', 'October 9, 1940'],
  ['Liverpool, England',
   'Childhood.',
   'The Beatles.',
   'Imaginative',
   'Creativity.'],
  ['Innovative songwriting, peace activism.'

In [10]:
claim_qa3.factoids

[['Paul McCartney is an iconic British musician.',
  'Paul McCartney is a singer.',
  'Paul McCartney is a songwriter.',
  'Paul McCartney is best known as a co-founder of The Beatles.',
  'The Beatles is one of the most influential bands in the history of music.',
  'Paul McCartney was born on June 18, 1942.',
  'Paul McCartney was born in Liverpool.',
  'Paul McCartney was born in England.',
  'Paul McCartney displayed a talent for music from a young age.',
  'Paul McCartney rose to fame alongside John Lennon.',
  'Paul McCartney rose to fame alongside George Harrison.',
  'Paul McCartney rose to fame alongside Ringo Starr.',
  'Paul McCartney rose to fame in the 1960s.',
  'Paul McCartney contributed to many classic hits.',
  'Paul McCartney contributed to the song "Hey Jude."',
  'Paul McCartney contributed to the song "Yesterday."',
  'Paul McCartney contributed to the song "Let It Be."',
  'Paul McCartney launched a successful solo career after The Beatles disbanded.',
  'Paul Mc

In [22]:
claim_qa3.response_fact_questions

[['\n    We are writing an answer to the question “write a paragraph about Paul McCartney”. So far we have written:\n    Paul McCartney is a renowned British musician, singer, songwriter, and multi-instrumentalist best known as a founding member of The Beatles, one of the most influential bands in the history of popular music. Born on June 18, 1942, in Liverpool, England, McCartney\'s musical career began in the late 1950s when he partnered with John Lennon, eventually leading to the formation of The Beatles in 1960. As one of the principal songwriters for the band, McCartney contributed timeless classics such as "Yesterday," "Hey Jude," and "Let It Be," showcasing his remarkable ability to blend melodies with innovative lyrics. Following the band\'s breakup in 1970, he embarked on a successful solo career and formed the band Wings, further exploring various musical styles. Beyond his musical achievements, McCartney is also known for his activism, particularly in the realms of animal r

In [14]:
stop - start

19.856560468673706

In [16]:
15 * 500 / 60

125.0

In [12]:
print("evaluate run time: ", stop - start)

In [17]:
result3.to_df()

,prompt,response,response_scores_exact_match,factoid_scores_exact_match
0,write a paragraph about Paul McCartney,"Paul McCartney is a renowned British musician,...",0.340741,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, ..."
1,write a paragraph about John Lennon,John Lennon was a seminal figure in music and ...,0.281818,"[0.0, 0.0, 0.2, 0.0, 1.0, 1.0, 0.0, 0.0, 0.8, ..."
